# Tutorial 1: Basics

This tutorial will talk about how to use this software from your own python project or Jupyter notebook.
There is also a nice command line interface that enables you to do the same with just two lines in your command line.

**NOTE FOR CONTRIBUTORS: Always clear all output before commiting (``Cell`` > ``All Output`` > ``Clear``)**!

In [1]:
# Magic
%matplotlib inline
# Reload modules whenever they change
%load_ext autoreload
%autoreload 2

# Make bclustering package available even without installation
import sys
sys.path = ["../../"] + sys.path

import bclustering

## Scanning

### Setting it up

Let's set up a scanner object and configure it.

In [2]:
s = bclustering.Scanner()

First we set up the function/distribution that we want to consider. Here we look into the branching ratio with respect to $q^2$ of $B\to D \,\tau\, \bar\nu_\tau$. The function of the differential branching ration is taken from the flavio package (https://flav-io.github.io/). The $q^2$ binning is chose to have 9 bins between $(3.2 \,\text{GeV})^2$ and $(11.6\,\text{GeV})^2$ and is implemented as follows

In [4]:
import flavio
import numpy as np

def dBrdq2(w, q):
    return flavio.sm_prediction("dBR/dq2(B+->Dtaunu)", q) + flavio.np_prediction("dBR/dq2(B+->Dtaunu)", w, q)

s.set_dfunction(
    dBrdq2,
    binning=np.linspace(3.2, 11.6, 10),
    normalize=True
)

First, let's set up the Wilson coefficients that need to be sampled. The Wilson coefficients are implemented using the Wilson package (https://wilson-eft.github.io/), which allows to use a variety of bases, EFTs and matches them to user specified scales.
Using the example of $B\longrightarrow D \tau \bar\nu_\tau$, we sample the coefficients ``CVL_bctaunutau``, ``CSL_bctaunutau`` and ``CT_bctaunutau`` from the ``flavio`` basis (https://wcxf.github.io/assets/pdf/WET.flavio.pdf) with 10 points between $-1$ and $1$ at the scale of 5 GeV:

In [6]:
s.set_wpoints_equidist(
    {
        "CVL_bctaunutau": (-1, 1, 10),
        "CSL_bctaunutau": (-1, 1, 10),
        "CT_bctaunutau": (-1, 1, 10)
    },
    scale=5,
    eft='WET',
    basis='flavio'
)

### Running it

In [7]:
d = bclustering.DataWithErrors()

In [8]:
s.run(d)

Scanner:INFO:Started queue with 1000 job(s) distributed over up to 8 core(s)/worker(s).
Scanning: 100%|████████████████████████| 1000/1000 [00:46<00:00, 21.67 wpoint/s]
Scanner:DEBUG:Converting data to pandas dataframe.
Scanner:INFO:Integration done.


The results are saved in a dataframe, ``d.df``. Let's have a look:

In [25]:
d.df.head()

,CVL_bctaunutau,CSL_bctaunutau,CT_bctaunutau,bin0,bin1,bin2,bin3,bin4,bin5,bin6,bin7,bin8,cluster,bpoint
index,,,,,,,,,,,,,,
0,-1.0,-1.0,-1.000000,0.019218,0.075804,0.124671,0.154968,0.166859,0.161631,0.140382,0.104140,0.052326,0,False
1,-1.0,-1.0,-0.777778,0.019774,0.077057,0.125625,0.155325,0.166726,0.161173,0.139682,0.103217,0.051420,0,False
2,-1.0,-1.0,-0.555556,0.020911,0.079837,0.127986,0.156359,0.166421,0.159859,0.137758,0.101089,0.049780,0,False
3,-1.0,-1.0,-0.333333,0.022681,0.084329,0.131975,0.158200,0.165919,0.157526,0.134387,0.097593,0.047389,0,False
4,-1.0,-1.0,-0.111111,0.024851,0.089976,0.137132,0.160655,0.165283,0.154420,0.129933,0.093153,0.044598,1,False


## Clustering

In [10]:
c = bclustering.cluster.HierarchyCluster(d)

In [11]:
c.set_metric()

In [12]:
c.build_hierarchy()

Scanner:DEBUG:Building hierarchy.
Scanner:DEBUG:Done


In [13]:
c.cluster(max_d=0.04)

Scanner:INFO:Performing clustering.
Scanner:INFO:Clustering resulted in 4 get_clusters.
Scanner:INFO:Done


In [14]:
c.write()

Let's take a look and notice the new column ``cluster``:

In [17]:
d.df.head()

,CVL_bctaunutau,CSL_bctaunutau,CT_bctaunutau,bin0,bin1,bin2,bin3,bin4,bin5,bin6,bin7,bin8,cluster
index,,,,,,,,,,,,,
0,-1.0,-1.0,-1.000000,0.019218,0.075804,0.124671,0.154968,0.166859,0.161631,0.140382,0.104140,0.052326,0
1,-1.0,-1.0,-0.777778,0.019774,0.077057,0.125625,0.155325,0.166726,0.161173,0.139682,0.103217,0.051420,0
2,-1.0,-1.0,-0.555556,0.020911,0.079837,0.127986,0.156359,0.166421,0.159859,0.137758,0.101089,0.049780,0
3,-1.0,-1.0,-0.333333,0.022681,0.084329,0.131975,0.158200,0.165919,0.157526,0.134387,0.097593,0.047389,0
4,-1.0,-1.0,-0.111111,0.024851,0.089976,0.137132,0.160655,0.165283,0.154420,0.129933,0.093153,0.044598,1


## Selecting benchmark points

In [18]:
b = bclustering.Benchmark(d)

In [19]:
b.set_metric()

In [20]:
b.select_bpoints()

In [21]:
b.write()

Let's take a look and notice the new column ``bpoint``:

In [22]:
d.df.head()

,CVL_bctaunutau,CSL_bctaunutau,CT_bctaunutau,bin0,bin1,bin2,bin3,bin4,bin5,bin6,bin7,bin8,cluster,bpoint
index,,,,,,,,,,,,,,
0,-1.0,-1.0,-1.000000,0.019218,0.075804,0.124671,0.154968,0.166859,0.161631,0.140382,0.104140,0.052326,0,False
1,-1.0,-1.0,-0.777778,0.019774,0.077057,0.125625,0.155325,0.166726,0.161173,0.139682,0.103217,0.051420,0,False
2,-1.0,-1.0,-0.555556,0.020911,0.079837,0.127986,0.156359,0.166421,0.159859,0.137758,0.101089,0.049780,0,False
3,-1.0,-1.0,-0.333333,0.022681,0.084329,0.131975,0.158200,0.165919,0.157526,0.134387,0.097593,0.047389,0,False
4,-1.0,-1.0,-0.111111,0.024851,0.089976,0.137132,0.160655,0.165283,0.154420,0.129933,0.093153,0.044598,1,False


## Preserving results

Now it's time to write out the results for later use.

In [27]:
d.write("output/cluster", "tutorial_basics", overwrite="overwrite")

DFMD:INFO:Will write dataframe to 'output/cluster/tutorial_basics_data.csv'.
DFMD:DEBUG:Done
DFMD:INFO:Will write metadata to 'output/cluster/tutorial_basics_metadata.json'.
DFMD:DEBUG:Done


This will not only write out the data itself, but also a lot of associated metadata that makes it easy to later reconstruct what the data actually represents. This was accumulated in the attribute ``d.md`` over all steps:

In [28]:
d.md

defaultdict(<function bclustering.util.metadata.nested_dict>,
            {'bpoint': defaultdict(<function bclustering.util.metadata.nested_dict>,
                         {'bpoint': defaultdict(<function bclustering.util.metadata.nested_dict>,
                                      {'cluster_column': defaultdict(<function bclustering.util.metadata.nested_dict>,
                                                   {}),
                                       'metric': defaultdict(<function bclustering.util.metadata.nested_dict>,
                                                   {'args': [],
                                                    'kwargs': {}})})}),
             'cluster': defaultdict(<function bclustering.util.metadata.nested_dict>,
                         {'cluster': defaultdict(<function bclustering.util.metadata.nested_dict>,
                                      {'cluster_args': {'max_d': 0.04},
                                       'git': {'branch': 'HEAD',
           

More about output files and metadata in the next part of the tutorial.